# Entity Recognition

Veamos las entidades en los títulos a ver qué anda dando vueltas...

In [51]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from mongoengine import connect
from hatespeech_models import Tweet, Article

client = connect("hatespeech-selection")
db = client["hatespeech-selection"]

Article.objects.count()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


68896

In [52]:
import spacy

nlp = spacy.load("es_core_news_sm")

In [35]:
selected_articles = Article.objects(selected=True)

In [53]:
selected_articles[0].first_paragraphs

'El Gobernador adelantó que, en poco tiempo, la aplicación Cuidar será de uso obligatorio en la Provincia\n\nEl gobernador bonaerense, Axel Kicillof , sostuvo hoy que los expertos epidemiológicos y sanitaristas recomiendan "fortalecer el aislamiento" social y recordó que a partir de hoy "el permiso para circular es obligatorio en la Provincia".\n\n"Nosotros vamos hacia la obligatoriedad de la aplicación Cuidar pero para eso hacen falta algunos ajustes", expresó Kicillof en declaraciones a radio Provincia.'

In [57]:
from tqdm.auto import tqdm
from collections import defaultdict

entities = defaultdict(list)

def normalize(entity):
    toks = entity.split(" ")
    toks = [t for t in toks if t.isalpha()]
    return " ".join(toks)

for article in tqdm(selected_articles):
    #import ipdb; ipdb.set_trace()
    current_entity = None
    current_type = None
    #print(article.title)
    for t in nlp(article.title):
        #print(t.text, t.ent_iob_)
        if (t.ent_iob_ == 'B' or t.ent_iob_ == 'O') and current_entity:
            entities[current_type].append(normalize(current_entity))
            current_type = None
            current_entity = None 
            
        if t.ent_iob_ == "B":
            current_entity = t.text
            current_type = t.ent_type_
        elif t.ent_iob_ == "I":
            current_entity += " " + t.text
    if current_entity:
        entities[current_type].append(normalize(current_entity))


In [58]:
entities.keys()



dict_keys(['PER', 'LOC', 'MISC', 'ORG'])

## PERSONAS

In [71]:
from collections import Counter

def show(ent_type):
    counter = Counter(t for t in entities[ent_type] if t)
    for i, p in enumerate(counter.most_common()):
        print(f"{i+1:<3}. {p[0]:<25} --> {p[1]}")


In [72]:
show("PER")

1  . Alberto Fernández         --> 86
2  . Coronavirus               --> 20
3  . Cristina Kirchner         --> 20
4  . Mauricio Macri            --> 18
5  . Macri                     --> 16
6  . Juan Grabois              --> 16
7  . Donald Trump              --> 15
8  . Florencia Kirchner        --> 14
9  . Sergio Berni              --> 14
10 . Santiago Cafiero          --> 13
11 . Ginés González García     --> 12
12 . Axel Kicillof             --> 11
13 . Presidente                --> 11
14 . Elisa Carrió              --> 11
15 . Cristina Pérez            --> 10
16 . Patricia Bullrich         --> 10
17 . Xi Jinping                --> 10
18 . Juana Viale               --> 10
19 . Devoto                    --> 9
20 . Jimena Barón              --> 9
21 . Qué                       --> 8
22 . Ofelia Fernández          --> 8
23 . Marcelo Tinelli           --> 7
24 . Viviana Canosa            --> 7
25 . Luis                      --> 7
26 . Eduardo Feinmann          --> 7
27 . Wuhan          

## LOC

In [73]:
show("LOC")

1  . China                     --> 122
2  . Argentina                 --> 80
3  . la Argentina              --> 67
4  . Gobierno                  --> 41
5  . Coronavirus               --> 29
6  . Ginés González García     --> 23
7  . Ciudad                    --> 15
8  . Estados Unidos            --> 15
9  . Buenos Aires              --> 13
10 . Venezuela                 --> 11
11 . Uruguay                   --> 10
12 . Brasil                    --> 10
13 . Bolivia                   --> 7
14 . Obelisco                  --> 6
15 . Provincia                 --> 6
16 . Cuba                      --> 6
17 . Florencia                 --> 6
18 . Quilmes                   --> 6
19 . Ezeiza                    --> 6
20 . en Estados Unidos         --> 6
21 . Villa                     --> 5
22 . Italia                    --> 5
23 . España                    --> 5
24 . Paraguay                  --> 5
25 . Cómo                      --> 5
26 . Aerolíneas Argentinas     --> 5
27 . Wuhan               

## ORG

In [74]:
show("ORG")

1  . OMS                       --> 42
2  . ONU                       --> 6
3  . More Rial                 --> 5
4  . Congreso                  --> 5
5  . Morena Rial               --> 4
6  . Time                      --> 3
7  . Justicia                  --> 3
8  . Senado                    --> 3
9  . Policía                   --> 3
10 . Carta                     --> 3
11 . FMI                       --> 2
12 . ministro de Salud         --> 2
13 . Amnistía Internacional    --> 2
14 . PJ                        --> 2
15 . IFE                       --> 2
16 . CGT                       --> 2
17 . Financial Times           --> 2
18 . Xi Jinping                --> 2
19 . Frente de Todos           --> 2
20 . Banco Nación              --> 2
21 . Mercosur                  --> 2
22 . Jonatan Viale             --> 2
23 . TikTok                    --> 2
24 . Banco Central             --> 2
25 . Google                    --> 2
26 . Sandra Pitta              --> 1
27 . Partido Comunista Chino   --> 1


## MISC

In [75]:

show("MISC")

1  . No                        --> 11
2  . Me                        --> 10
3  . Tenemos                   --> 6
4  . Gobierno                  --> 5
5  . Si                        --> 5
6  . Estoy                     --> 4
7  . Elisa Carrió              --> 4
8  . Cristina                  --> 4
9  . AMBA                      --> 3
10 . Soy                       --> 3
11 . Presidente                --> 3
12 . Quinta de Olivos          --> 3
13 . Acá                       --> 3
14 . Organización Mundial de la Salud --> 3
15 . expropiación de Vicentin  --> 3
16 . WhatsApp                  --> 3
17 . Laura Azcurra             --> 3
18 . Casados                   --> 3
19 . Salud                     --> 2
20 . América Latina            --> 2
21 . Se                        --> 2
22 . Nadie                     --> 2
23 . Twitter                   --> 2
24 . Cualquier                 --> 2
25 . Tengo                     --> 2
26 . Policía Bonaerense        --> 2
27 . Están                   